In [24]:
import torch.nn as nn
import torch
import math
import torch.nn.functional as F
from sklearn import svm
from sklearn.model_selection  import train_test_split
from sklearn.datasets import load_digits
import copy
import numpy as np
import os
from sklearn.preprocessing import Normalizer,StandardScaler,MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,VarianceThreshold
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from torch import Tensor
from torch_geometric.nn import DenseGraphConv
import pickle
import torch.utils.data as Data
from torch.utils.data import DataLoader
path = "Project for ML/MoleculeEvaluationData/"

In [25]:
def load_from(path):
    f= open(path,'rb')
    #data = np.unpackbits(f)
    d = pickle.load(f)
    data = d["packed_fp"]
    label = d["values"]
    f.close()
    return label.squeeze().numpy(),np.unpackbits(data, axis=1)

In [26]:
train_label,train_data = load_from(path + "train.pkl")
test_label,test_data = load_from(path + "test.pkl")

In [142]:
def retrain(train_data,train_label,Num = 100000):
    train_data2 = np.zeros([Num,2048 * 2])
    train_label2 = np.zeros(Num)
    for i in range(Num):
        rnd1,rnd2 = np.random.randint(0,train_data.shape[0]),np.random.randint(0,train_data.shape[0])
        train_data2[i] = np.concatenate([train_data[rnd1],train_data[rnd2]])
            #        F.cosine_similarity(torch.tensor(train_data[rnd1].astype(float)),torch.tensor(train_data[rnd2].astype(float)),dim = 0).numpy()])
        train_label2[i] = train_label[rnd1] + train_label[rnd2]
    return train_data2,train_label2

In [43]:
TOTAL_NUM = 5
def cos_matrix(data):
    ret = np.zeros([TOTAL_NUM,TOTAL_NUM])
    for i in range(TOTAL_NUM):
        for j in range(TOTAL_NUM):
            ret[i][j] = F.cosine_similarity(torch.tensor(data[i].astype(float)),torch.tensor(data[j].astype(float)),dim = 0).numpy()
    return ret
def retrain2(train_data,train_label,Num = 1000):
    train_data2 = np.zeros([Num,TOTAL_NUM,2048])
    train_label2 = np.zeros([Num,TOTAL_NUM])
    ajmtx = np.zeros([Num,TOTAL_NUM,TOTAL_NUM])
    for i in range(Num):
        for j in range(TOTAL_NUM):
            rnd1 = np.random.randint(0,train_data.shape[0])
            train_data2[i][j] = train_data[rnd1]
            train_label2[i][j] = train_label[rnd1]
        ajmtx[i] = cos_matrix(train_data2[i])
        if i % 1000 == 0:
            print(i)
    return train_data2,train_label2,ajmtx

In [10]:
TOTAL_NUM = 10
def retrain(train_data,train_label,Num = 10000):
    train_data2 = np.zeros([Num,TOTAL_NUM * 2048])
    train_label2 = np.zeros(Num)
    for i in range(Num):
        x = 0
        for j in range(TOTAL_NUM):
            rnd1 = np.random.randint(0,train_data.shape[0])
            if j == 0:
                x = train_data[rnd1]
            else:
                x = np.concatenate([x,train_data[rnd1]])
            train_label2[i] = train_label2[i] + train_label[rnd1]
        train_data2[i] = x
    return train_data2,train_label2

In [4]:
Sel = PCA(n_components=128, copy=True, whiten=False)
#scaler = StandardScaler()
tdata = Sel.fit_transform(train_data)
sdata = Sel.transform(test_data)

In [5]:
tlabel = train_label
slabel = test_label

In [5]:
tdata = train_data
sdata = test_data

In [12]:
tdata,tlabel = retrain(train_data,train_label,4 * 10000)
sdata,slabel = retrain(test_data,test_label)

In [44]:
tdata,tlabel,tajmtx = retrain2(train_data,train_label,4 * 5000)
sdata,slabel,sajmtx = retrain2(test_data,test_label,5000)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
0
1000
2000
3000
4000


In [11]:
print(tajmtx.shape)

(40000, 10, 10)


In [ ]:
model = MLPRegressor(hidden_layer_sizes= (20,5), activation='relu',solver='adam',max_iter=500,
                    learning_rate_init=0.01,learning_rate='adaptive',early_stopping=False,epsilon = 1e-5,beta_1 = 0.9,verbose = True,
                     #tol = 1e-4,
                    )
model.fit(tdata,tlabel)

Iteration 1, loss = 313.98475611
Iteration 2, loss = 304.22807803
Iteration 3, loss = 295.21666312
Iteration 4, loss = 286.45363101
Iteration 5, loss = 277.90643887
Iteration 6, loss = 269.58321715
Iteration 7, loss = 261.48073621
Iteration 8, loss = 253.58819334
Iteration 9, loss = 245.90245460
Iteration 10, loss = 238.43187832
Iteration 11, loss = 231.15528901
Iteration 12, loss = 224.07983506
Iteration 13, loss = 217.20015712
Iteration 14, loss = 210.51481469
Iteration 15, loss = 204.00804276
Iteration 16, loss = 197.69385580
Iteration 17, loss = 191.55384933
Iteration 18, loss = 185.60194419
Iteration 19, loss = 179.80699877
Iteration 20, loss = 174.19036613
Iteration 21, loss = 168.74695055
Iteration 22, loss = 163.45581663
Iteration 23, loss = 158.33535115
Iteration 24, loss = 153.36538658
Iteration 25, loss = 148.54585384
Iteration 26, loss = 143.88890257
Iteration 27, loss = 139.36997390
Iteration 28, loss = 135.00033995
Iteration 29, loss = 130.77299819
Iteration 30, loss = 12

In [ ]:
z = model.predict(sdata)
zt = model.predict(tdata)
rate = (np.mean(abs(zt-tlabel)))
print('偏移率:',rate)
rate = (np.mean(abs(z-slabel)))
print('误差率:',rate)
rate = (np.sum(z-slabel))
print('全误差:',rate)
rate = (np.sum(slabel))
print('总和:',rate)

In [147]:
sdata,slabel = retrain(test_data,test_label,50000)

In [21]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1,16,5,1,2),
            nn.BatchNorm1d(16),
            nn.Sigmoid(),
            nn.MaxPool1d(2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(16,32,5,1,2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.out = nn.Sequential(
            nn.Linear(1024, 1),
            #nn.Dropout(),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

def save_model(net, path):
    torch.save(net, path)

def load_model(path):
    net = torch.load(path)
    return net
def train_model(net, data, label, lr, batch_size, epoch,early_stop = 10):
    # net = net.cuda()
    data = torch.Tensor(data)
    data = data.unsqueeze(1)
    label = torch.Tensor(label).float()
    # data =data.cuda()
    # label=label.cuda()
    
    train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.1, random_state=0)
    
    LR = lr
    BATCH_SIZE = batch_size
    EPOCH = epoch
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)

    train_dataset = Data.TensorDataset(train_data, train_label)
    train_loader = Data.DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )

    test_dataset = Data.TensorDataset(test_data, test_label)
    test_loader = Data.DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, LR, epochs=EPOCH, steps_per_epoch=len(train_loader))
    
    best = 1000
    be = 0
    for epoch in range(EPOCH):
        for step, (batch_data, batch_label) in enumerate(train_loader):
            if step % 1000 == 0:
                print('Epoch:', epoch + 1, '/', EPOCH, 'Step:', step)
            prediction = net(batch_data).squeeze()
            loss = F.l1_loss(prediction,batch_label)#F.cross_entropy(prediction, batch_label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        accuracy = []
        for stp, (test_x, test_y) in enumerate(test_loader):
            test_output = net(test_x).squeeze()
            accuracy.append(np.mean(abs(test_output-test_y).detach().numpy()))
        succ = np.mean(accuracy)
        if best > succ:
            best = succ
            save_model(net, "tmp_cnn.pkl")
            be = epoch
        if epoch - be >= early_stop:
            net = load_model("tmp_cnn.pkl")
            print('Early Stop in Epoch', epoch + 1, '| train loss:%.4f' % loss, '| test accuracy:%.4f' % succ)
            accuracy = []
            for stp, (test_x, test_y) in enumerate(test_loader):
                test_output = net(test_x).squeeze()
                accuracy.append(np.mean(abs(test_output-test_y).detach().numpy()))
            succ = np.mean(accuracy)
            print('Best Epoch', be + 1, '| test accuracy:%.4f' % succ)
            return net
        print('Epoch', epoch + 1, '| train loss:%.4f' % loss, '| test accuracy:%.4f' % succ)
    return net

def test_model(net,data,label):
    test_data = torch.Tensor(data).unsqueeze(1)
    test_label = torch.Tensor(label).float()
    
    test_dataset = Data.TensorDataset(test_data,test_label)
    test_loader = Data.DataLoader(
        dataset=test_dataset,
        batch_size=32,
        shuffle=True,
    )

    ret = []
    ret2 = []
    ret3 = []
    for stp, (test_x,test_y) in enumerate(test_loader):
        pred_y = net(test_x).squeeze()
        ret.append(np.mean(np.mean(abs(pred_y-test_y).detach().numpy())))
        ret2.append(np.sum(np.sum((pred_y-test_y).detach().numpy())))
        ret3.append(np.sum(np.sum((test_y).detach().numpy())))
    #print(test_y)
    #print(pred_y)
    
    print("误差：",np.mean(ret))
    print("全误差：",np.sum(ret2))
    print("总值：",np.sum(ret3))
    return np.mean(ret)

In [22]:
i,j = 1,1
td = tdata
tl = tlabel
ttd = sdata
ttl = slabel
cnn = CNN()
cnn = train_model(cnn,td,tl,lr=0.01, batch_size=128, epoch=30)
save_model(cnn, "cnn_"+str(i+1)+"_"+str(j+1)+"_.pkl")

Epoch: 1 / 30 Step: 0
Epoch: 1 / 30 Step: 1000
Epoch: 1 / 30 Step: 2000
Epoch 1 | train loss:0.4790 | test accuracy:0.4533
Epoch: 2 / 30 Step: 0
Epoch: 2 / 30 Step: 1000
Epoch: 2 / 30 Step: 2000
Epoch 2 | train loss:0.4469 | test accuracy:0.4502
Epoch: 3 / 30 Step: 0
Epoch: 3 / 30 Step: 1000
Epoch: 3 / 30 Step: 2000
Epoch 3 | train loss:0.6647 | test accuracy:0.4474
Epoch: 4 / 30 Step: 0
Epoch: 4 / 30 Step: 1000
Epoch: 4 / 30 Step: 2000
Epoch 4 | train loss:0.5196 | test accuracy:0.4457
Epoch: 5 / 30 Step: 0
Epoch: 5 / 30 Step: 1000
Epoch: 5 / 30 Step: 2000
Epoch 5 | train loss:0.6862 | test accuracy:0.4475
Epoch: 6 / 30 Step: 0
Epoch: 6 / 30 Step: 1000
Epoch: 6 / 30 Step: 2000
Epoch 6 | train loss:0.4250 | test accuracy:0.4457
Epoch: 7 / 30 Step: 0
Epoch: 7 / 30 Step: 1000
Epoch: 7 / 30 Step: 2000
Epoch 7 | train loss:0.3711 | test accuracy:0.4464
Epoch: 8 / 30 Step: 0
Epoch: 8 / 30 Step: 1000
Epoch: 8 / 30 Step: 2000
Epoch 8 | train loss:0.3299 | test accuracy:0.4470
Epoch: 9 / 30 St

In [23]:
test_model(cnn, ttd, ttl)

误差： 1.8902924
全误差： -233116.9
总值： 241742.94


1.8902924

In [38]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = DenseGraphConv(in_channels, hidden_channels)
        self.conv2 = DenseGraphConv(hidden_channels, out_channels)

    def forward(self, x: Tensor, adj: Tensor) -> Tensor:
        x = self.conv1(x, adj).relu()
        x = self.conv2(x, adj)
        return x

def save_model(net, path):
    torch.save(net, path)

def load_model(path):
    net = torch.load(path)
    return net

In [39]:
def train_model(net, data,label,ajmtx,lr, batch_size, epoch,early_stop = 10):
    # net = net.cuda()
    data = torch.Tensor(data)
    ajmtx = torch.Tensor(ajmtx)
    label = torch.Tensor(label).float()
    # data =data.cuda()
    # label=label.cuda()
    
    #train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.1, random_state=0)
    
    LR = lr
    BATCH_SIZE = batch_size
    EPOCH = epoch
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)

    train_dataset = Data.TensorDataset(data,ajmtx,label)
    train_loader = Data.DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )

    #test_dataset = Data.TensorDataset(test_data,aj_data,test_label)
    test_loader = Data.DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, LR, epochs=EPOCH, steps_per_epoch=len(train_loader))
    
    best = 1000
    be = 0
    for epoch in range(EPOCH):
        for step, (batch_data,batch_aj,batch_label) in enumerate(train_loader):
            if step % 100 == 0:
                print('Epoch:', epoch + 1, '/', EPOCH, 'Step:', step)
            prediction = net(batch_data,batch_aj).squeeze()
            loss = F.mse_loss(prediction,batch_label)#F.cross_entropy(prediction, batch_label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        accuracy = []
        for stp, (test_x,test_aj,test_y) in enumerate(test_loader):
            test_output = net(test_x,test_aj).squeeze()
            accuracy.append(np.mean(np.mean(abs(test_output-test_y).detach().numpy())))
        succ = np.mean(accuracy)
        if best > succ:
            best = succ
            save_model(net, "tmp_gcn.pkl")
            be = epoch
        if epoch - be >= early_stop:
            net = load_model("tmp_gcn.pkl")
            print('Early Stop in Epoch', epoch + 1, '| train loss:%.4f' % loss, '| test accuracy:%.4f' % succ)
            accuracy = []
            for stp, (test_x,test_aj,test_y) in enumerate(test_loader):
                test_output = net(test_x,test_aj).squeeze()
                accuracy.append(np.mean(np.mean(abs(test_output-test_y).detach().numpy())))
            succ = np.mean(accuracy)
            print('Best Epoch', be + 1, '| test accuracy:%.4f' % succ)
            return net
        print('Epoch', epoch + 1, '| train loss:%.4f' % loss, '| test accuracy:%.4f' % succ)
    return net

In [40]:
def test_model(net,data,label,ajmtx):
    test_data = torch.Tensor(data)
    ajmtx = torch.Tensor(ajmtx)
    test_label = torch.Tensor(label).float()
    
    test_dataset = Data.TensorDataset(test_data,ajmtx,test_label)
    test_loader = Data.DataLoader(
        dataset=test_dataset,
        batch_size=32,
        shuffle=True,
    )

    ret = []
    ret2 = []
    ret3 = []
    for stp, (test_x,test_aj,test_y) in enumerate(test_loader):
        pred_y = net(test_x,test_aj).squeeze()
        ret.append(np.mean(np.mean(abs(pred_y-test_y).detach().numpy())))
        ret2.append(np.sum(np.sum((pred_y-test_y).detach().numpy())))
        ret3.append(np.sum(np.sum((test_y).detach().numpy())))
    #print(test_y)
    #print(pred_y)
    
    print("误差：",np.mean(ret))
    print("全误差：",np.sum(ret2))
    print("总值：",np.sum(ret3))
    return np.mean(ret)

In [45]:
td = tdata
tl = tlabel
ttd = sdata
ttl = slabel
tj = tajmtx
gcn = GCN(2048, 16, 1)
gcn = train_model(gcn,td,tl,tj,lr=0.01, batch_size=32, epoch=300)
save_model(gcn, "gcn_4.pkl")

Epoch: 1 / 300 Step: 0
Epoch: 1 / 300 Step: 100
Epoch: 1 / 300 Step: 200
Epoch: 1 / 300 Step: 300
Epoch: 1 / 300 Step: 400
Epoch: 1 / 300 Step: 500
Epoch: 1 / 300 Step: 600
Epoch 1 | train loss:1.0700 | test accuracy:0.5858
Epoch: 2 / 300 Step: 0
Epoch: 2 / 300 Step: 100
Epoch: 2 / 300 Step: 200
Epoch: 2 / 300 Step: 300
Epoch: 2 / 300 Step: 400
Epoch: 2 / 300 Step: 500
Epoch: 2 / 300 Step: 600
Epoch 2 | train loss:1.1031 | test accuracy:0.5835
Epoch: 3 / 300 Step: 0
Epoch: 3 / 300 Step: 100
Epoch: 3 / 300 Step: 200
Epoch: 3 / 300 Step: 300
Epoch: 3 / 300 Step: 400
Epoch: 3 / 300 Step: 500
Epoch: 3 / 300 Step: 600
Epoch 3 | train loss:0.8024 | test accuracy:0.5129
Epoch: 4 / 300 Step: 0
Epoch: 4 / 300 Step: 100
Epoch: 4 / 300 Step: 200
Epoch: 4 / 300 Step: 300
Epoch: 4 / 300 Step: 400
Epoch: 4 / 300 Step: 500
Epoch: 4 / 300 Step: 600
Epoch 4 | train loss:1.1399 | test accuracy:0.5024
Epoch: 5 / 300 Step: 0
Epoch: 5 / 300 Step: 100
Epoch: 5 / 300 Step: 200
Epoch: 5 / 300 Step: 300
Epoch:

Epoch: 36 / 300 Step: 600
Epoch 36 | train loss:0.3532 | test accuracy:0.3596
Epoch: 37 / 300 Step: 0
Epoch: 37 / 300 Step: 100
Epoch: 37 / 300 Step: 200
Epoch: 37 / 300 Step: 300
Epoch: 37 / 300 Step: 400
Epoch: 37 / 300 Step: 500
Epoch: 37 / 300 Step: 600
Epoch 37 | train loss:0.5233 | test accuracy:0.3435
Epoch: 38 / 300 Step: 0
Epoch: 38 / 300 Step: 100
Epoch: 38 / 300 Step: 200
Epoch: 38 / 300 Step: 300
Epoch: 38 / 300 Step: 400
Epoch: 38 / 300 Step: 500
Epoch: 38 / 300 Step: 600
Epoch 38 | train loss:0.2778 | test accuracy:0.3612
Epoch: 39 / 300 Step: 0
Epoch: 39 / 300 Step: 100
Epoch: 39 / 300 Step: 200
Epoch: 39 / 300 Step: 300
Epoch: 39 / 300 Step: 400
Epoch: 39 / 300 Step: 500
Epoch: 39 / 300 Step: 600
Epoch 39 | train loss:0.9953 | test accuracy:0.3559
Epoch: 40 / 300 Step: 0
Epoch: 40 / 300 Step: 100
Epoch: 40 / 300 Step: 200
Epoch: 40 / 300 Step: 300
Epoch: 40 / 300 Step: 400
Epoch: 40 / 300 Step: 500
Epoch: 40 / 300 Step: 600
Epoch 40 | train loss:0.4496 | test accuracy:0

Epoch: 72 / 300 Step: 100
Epoch: 72 / 300 Step: 200
Epoch: 72 / 300 Step: 300
Epoch: 72 / 300 Step: 400
Epoch: 72 / 300 Step: 500
Epoch: 72 / 300 Step: 600
Epoch 72 | train loss:0.2699 | test accuracy:0.3143
Epoch: 73 / 300 Step: 0
Epoch: 73 / 300 Step: 100
Epoch: 73 / 300 Step: 200
Epoch: 73 / 300 Step: 300
Epoch: 73 / 300 Step: 400
Epoch: 73 / 300 Step: 500
Epoch: 73 / 300 Step: 600
Epoch 73 | train loss:0.4186 | test accuracy:0.3183
Epoch: 74 / 300 Step: 0
Epoch: 74 / 300 Step: 100
Epoch: 74 / 300 Step: 200
Epoch: 74 / 300 Step: 300
Epoch: 74 / 300 Step: 400
Epoch: 74 / 300 Step: 500
Epoch: 74 / 300 Step: 600
Epoch 74 | train loss:0.3681 | test accuracy:0.3321
Epoch: 75 / 300 Step: 0
Epoch: 75 / 300 Step: 100
Epoch: 75 / 300 Step: 200
Epoch: 75 / 300 Step: 300
Epoch: 75 / 300 Step: 400
Epoch: 75 / 300 Step: 500
Epoch: 75 / 300 Step: 600
Epoch 75 | train loss:0.3867 | test accuracy:0.3128
Epoch: 76 / 300 Step: 0
Epoch: 76 / 300 Step: 100
Epoch: 76 / 300 Step: 200
Epoch: 76 / 300 Step

Epoch: 107 / 300 Step: 200
Epoch: 107 / 300 Step: 300
Epoch: 107 / 300 Step: 400
Epoch: 107 / 300 Step: 500
Epoch: 107 / 300 Step: 600
Early Stop in Epoch 107 | train loss:0.2203 | test accuracy:0.2740
Best Epoch 97 | test accuracy:0.2738


In [46]:
test_model(gcn,ttd,ttl,sajmtx)
test_model(gcn,td,tl,tajmtx)

误差： 1.8956462
全误差： -37512.883
总值： 48418.805
误差： 0.27378988
全误差： -4702.0825
总值： 45462.54


0.27378988